# Data Modification
This script modifies the data in a couple of ways

### Handle cancelled flights
This script assigns delay values to cancelled flights. It does so in different ways:
* Delayed flights get a delay of 12h
* Delayed flights get a delay of 24h
* Delayed flights get as their delay the time until the next flight. If it does not exist, it will get assigned a delay of 12h.

### Aggregate delay columns into 'Total_Delay'
As the customer does not care about the origin of the delay but rather the total delay, it is not important to predict the delay originating from different sources as a whole.

### Remove unrelated columns/obsolete columns
From a qualitative point of view, it is obvious that some variables contain no predictive power. These columns will be removed. Besides that, some columns are linear combinations of others and therefore do not contain any more information. These will also be removed.

### Reformat datetimes
The dates are represented as strings. We will convert them to datetimes. Moreover, columns containing the day of week and month will be added. We intent to use these instead of the data for predictions. A column of an indicator variable that indicates whether the day is a hollyday will also be added.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import re

airlines = pd.read_csv('Data\\airlines.csv')
airports = pd.read_csv('Data\\airports.csv')
past_data = pd.read_csv('Data\\historic_data.csv')
august_data = pd.read_csv('Data\\future_data.csv')

In [2]:
# Display original data
past_data.head()

df = past_data

# Randomly sample a small data set from past_data to work on for testing.
# df = past_data.sample(n=300, replace=False)
# df.reset_index(inplace=True, drop=True)

## Create a Total_Delay column

In [3]:
DelayCol = [col for col in df.columns.values if "DELAY" in col]
df['TOTAL_DELAY'] = df[DelayCol].sum(axis=1, skipna=True)

df.head()

,SCHEDULED_DEPARTURE,DEPARTURE_TIME,SCHEDULED_ARRIVAL,ARRIVAL_TIME,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,...,WHEELS_ON,TAXI_IN,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,TOTAL_DELAY
0,2019-03-01 00:01:00,00:02:00,01:37:00,01:20:00,UA,1238,N812UA,DEN,BIL,1.0,...,117.0,3.0,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2019-03-01 00:01:00,00:29:00,01:33:00,01:52:00,EV,6134,N14171,ORD,BNA,28.0,...,144.0,8.0,0,NaN,0.0,0.0,19.0,0.0,0.0,47.0
2,2019-03-01 00:01:00,NaN,01:53:00,NaN,EV,4682,N15910,ORD,BHM,NaN,...,NaN,NaN,1,B,NaN,NaN,NaN,NaN,NaN,0.0
3,2019-03-01 00:02:00,23:59:00,03:04:00,02:56:00,UA,1550,N822UA,ORD,BTV,-3.0,...,252.0,4.0,0,NaN,NaN,NaN,NaN,NaN,NaN,-3.0
4,2019-03-01 00:02:00,00:36:00,03:22:00,03:30:00,UA,1502,N33262,ORD,BOS,34.0,...,326.0,4.0,0,NaN,NaN,NaN,NaN,NaN,NaN,34.0


## Drop unnecessary columns
We drop the columns that have no predictive power from a qualitative point of view. Furthermore, as our goal is to predict the delay of flights in the future, we cannot use variables that can only be recorded during or after the flight. These columns are also dropped.

In [4]:
df.columns.values

array(['SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'SCHEDULED_ARRIVAL',
       'ARRIVAL_TIME', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'DEPARTURE_DELAY',
       'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME',
       'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'CANCELLED',
       'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY',
       'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY',
       'TOTAL_DELAY'], dtype=object)

In [5]:
df.drop(columns=['DEPARTURE_TIME', 'ARRIVAL_TIME', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'DEPARTURE_DELAY', 'TAXI_OUT', 'TAXI_IN',
                 'WHEELS_OFF', 'WHEELS_ON', 'AIR_TIME', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 
                 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], inplace=True)
df.head()

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,CANCELLED,TOTAL_DELAY
0,2019-03-01 00:01:00,01:37:00,UA,DEN,BIL,96.0,78.0,455,0,1.0
1,2019-03-01 00:01:00,01:33:00,EV,ORD,BNA,92.0,83.0,409,0,47.0
2,2019-03-01 00:01:00,01:53:00,EV,ORD,BHM,112.0,NaN,584,1,0.0
3,2019-03-01 00:02:00,03:04:00,UA,ORD,BTV,122.0,117.0,763,0,-3.0
4,2019-03-01 00:02:00,03:22:00,UA,ORD,BOS,140.0,114.0,867,0,34.0


## Reformat datetimes
Since dateparts are not mentioned in some columns, we guess them based on the columns that do mention them. 

In [6]:
# Convert string values that indicate dates and times to datetime objects
toConvert = ['SCHEDULED_DEPARTURE']
for col in toConvert:
    df[col] = pd.to_datetime(df[col])

In [7]:
df.head()

,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,CANCELLED,TOTAL_DELAY
0,2019-03-01 00:01:00,01:37:00,UA,DEN,BIL,96.0,78.0,455,0,1.0
1,2019-03-01 00:01:00,01:33:00,EV,ORD,BNA,92.0,83.0,409,0,47.0
2,2019-03-01 00:01:00,01:53:00,EV,ORD,BHM,112.0,NaN,584,1,0.0
3,2019-03-01 00:02:00,03:04:00,UA,ORD,BTV,122.0,117.0,763,0,-3.0
4,2019-03-01 00:02:00,03:22:00,UA,ORD,BOS,140.0,114.0,867,0,34.0


## Handle cancelled flights

In [8]:
def set_delay_12h(row):
    MINUTES_IN_HOUR = 60
    HOURS = 12
    if row['CANCELLED'] == 1:
        return MINUTES_IN_HOUR*HOURS
    else:
        return row['TOTAL_DELAY']

def set_delay_24h(row):
    MINUTES_IN_HOUR = 60
    HOURS = 24
    if row['CANCELLED'] == 1:
        return MINUTES_IN_HOUR*HOURS
    else:
        return row['TOTAL_DELAY']

def set_delay_next_flight(row):
    MINUTES_IN_HOUR = 60
    HOURS = 12
    if row['CANCELLED'] == 1:
        same_flights = df.loc[(df['ORIGIN_AIRPORT'] == row['ORIGIN_AIRPORT']) & (df['DESTINATION_AIRPORT'] == row['DESTINATION_AIRPORT'])]
        same_flights[same_flights['SCHEDULED_DEPARTURE'] > row['SCHEDULED_DEPARTURE']].sort_values(by='SCHEDULED_DEPARTURE', inplace=True)
        
        if same_flights.shape[0] > 0:
            # print(dt.datetime.fromtimestamp(same_flights.at[1, 'SCHEDULED_DEPARTURE']))
            
            print("2nd time", type(row['SCHEDULED_DEPARTURE']))
            print(same_flights.at[0, 'SCHEDULED_DEPARTURE'] - row['SCHEDULED_DEPARTURE'])
            return (same_flights.at[0, 'SCHEDULED_DEPARTURE'] - row['SCHEDULED_DEPARTURE']).seconds / 60
        else:
            return HOURS*MINUTES_IN_HOUR
    else:
        return row['TOTAL_DELAY']

df['TARGET_1'] = df.apply(lambda x: set_delay_12h(x), axis = 1)
df['TARGET_2'] = df.apply(lambda x: set_delay_24h(x), axis = 1)
# df['TARGET_3'] = df.apply(lambda x: set_delay_next_flight(x), axis = 1)

In [9]:
df.to_csv("Data/modified_training_data.csv")